Extraction des données : Profils tranverses de la vitesse axiale

In [3]:
import pandas as pd
import re
import matplotlib.pyplot as plt

In [10]:
def load_data(file_path):
    dataframes = []
    x_values = []  # List to store the x values
    with open(file_path, 'r') as file:
        lines = file.readlines()
        x_value = None
        data = []
        for line in lines:
            if 'xy/key/label' in line:
                match = re.search('line-(.*)m', line)
                if match is not None:
                    x_value = float(match.group(1))
                    x_values.append(x_value)  # Append the x value to the list
            elif line.strip() == ')':
                df = pd.DataFrame(data, columns=['x', 'y', 'u(y)'])
                df = df.sort_values(by='y')
                dataframes.append(df)
                data = []
            elif '\t' in line:
                y, u_y = map(float, line.split('\t'))
                data.append([x_value, y, u_y])
    # Concatenate all dataframes into one
    final_df = pd.concat(dataframes)
    return final_df, x_values  # Return the dataframe and the list of x values

file_path = 'data/ProfilsTransverseVitesseAxiale'
data, x_values = load_data(file_path)
print(data)
print(x_values)

       x         y      u(y)
0    0.0  0.000000  15.00000
1    0.0  0.001245  15.00000
2    0.0  0.002416  15.00000
3    0.0  0.003517  15.00000
4    0.0  0.004553  15.00000
..   ...       ...       ...
196  0.5  0.092784   4.60054
197  0.5  0.094527   4.60070
198  0.5  0.096310   4.60083
199  0.5  0.098134   4.60092
200  0.5  0.100000   4.60096

[1210 rows x 3 columns]
[0.0, 0.1, 0.2, 0.3, 0.4, 0.5]


On cherche à calculer $\delta(x)$ l'épaisseur de la couche de cisaillement.

La couche de cisaillement est ici définie comme la zone où $\overline{u}$ varie de $U_0 + 0,1 \Delta U$ à $U_0 + 0,9 \Delta U$ où $\Delta U = U_1 - U_0$.

In [ ]:
U_0 = 5
U_1 = 15

In [12]:
def shear_layer_thickness(df, x, _U_0 = U_0, _U_1 = U_1):
    # Filter dataframe for given x
    df_x = df[df['x'] == x]
    
    # Calculate delta_U
    #U_0 = df_x['u(y)'].min()
    #U_1 = df_x['u(y)'].max()
    delta_U = _U_1 - _U_0
    
    # Filter dataframe for u(y) values in shear layer
    df_shear = df_x[(df_x['u(y)'] >= _U_0 + 0.1*delta_U) & (df_x['u(y)'] <= _U_0 + 0.9*delta_U)]
    # Calculate and return shear layer thickness
    thickness = df_shear['y'].max() - df_shear['y'].min()
    return thickness

# Test the function
for x in x_values:
    thickness = shear_layer_thickness(data, x)
    print(f"Shear layer thickness at x = {x}: {thickness}")




Shear layer thickness at x = 0.0: 0.0
Shear layer thickness at x = 0.1: 0.007730499999999998
Shear layer thickness at x = 0.2: 0.014853499999999999
Shear layer thickness at x = 0.3: 0.0223533
Shear layer thickness at x = 0.4: 0.02944563
Shear layer thickness at x = 0.5: 0.037416290000000005


De même on va calculer $y^*$, qui à un $x$ donné est l'ordonnée $y$ telle que $\overline{u}(y^*) = \dfrac{U_0 + U_1}{2}$

In [14]:
def compute_y_star(df, x, _U_0 = U_0, _U_1 = U_1):
    # Filter dataframe for given x
    df_x = df[df['x'] == x]
    
    # Calculate the midpoint of U_0 and U_1
    U_mid = (_U_0 + _U_1) / 2
    
    # Find the y value where U(y) is closest to U_mid
    y_star = df_x.iloc[(df_x['u(y)'] - U_mid).abs().idxmin()]['y']
    
    return y_star

# Test the function
for x in x_values:
    y_star = compute_y_star(data, x)
    print(f"y_star at x={x}: {y_star}")

y_star at x=0.0: 0.02
y_star at x=0.1: 0.0211001
y_star at x=0.2: 0.0217576
y_star at x=0.3: 0.0250053
y_star at x=0.4: 0.0220731
y_star at x=0.5: 0.0215589


Maintenant on peut adimensionnaliser les profils, avec :
- l'abscisse $\dfrac{y-y^*}{\delta}$

- l'ordonnée $\dfrac{\overline{u} - U_0}{U_1 - U_0}$